In [1]:
import os
import numpy as np
import onnxruntime as rt
import onnx
import matplotlib.pyplot as plt
from train.dataloader import CommaDataset, BatchDataLoader, BackgroundGenerator
from torch.utils.data import DataLoader
import cv2
from common.transformations.camera import eon_intrinsics, FULL_FRAME_SIZE as eon_f_frame_size, get_view_frame_from_calib_frame


In [2]:
plot_img_height, plot_img_width = 480, 640 # can be reduced for wandb
width, height = eon_f_frame_size

_BB_SCALE = eon_f_frame_size[0] / plot_img_width
calib_scale = eon_f_frame_size[0] / plot_img_width
_BB_TO_FULL_FRAME = np.asarray([
    [_BB_SCALE, 0., 0.],
    [0., _BB_SCALE, 0.],
    [0., 0., 1.]])
_CALIB_BB_TO_FULL = np.asarray([
    [calib_scale, 0., 0.],
    [0., calib_scale, 0.],
    [0., 0., 1.]])

PATH = 4955
LANE_LINES = PATH+528
LANE_LINE_PROB = LANE_LINES+8
ROAD_EDGES = LANE_LINE_PROB+264
LEADS = ROAD_EDGES+102
LEAD_PROB = LEADS+3
DESIRE_STATE = LEAD_PROB+8
META = DESIRE_STATE+80
POSE = META+12
RECURRENT_STATE = POSE+512

TRAJECTORY_SIZE = 33
MAX_DISTANCE = 140.
LANE_OFFSET = 1.8
MAX_REL_V = 10.

LEAD_X_SCALE = 10
LEAD_Y_SCALE = 10
PLAN_MEAN = 0
PLAN_STD = 1
PLAN_X = 0
PLAN_Y = 1
LANE_MEAN = 0
LANE_Y = 0

In [3]:
def seperate_points_and_std_values(path):
    points_indices = np.arange(0, path.shape[-1], 2)
    std_indices = np.arange(1, path.shape[-1], 2)

    points = path[:, points_indices]
    std_values = path[:, std_indices]

    return points, std_values


def extract_preds(res):
    # N is batch_size

    plan_start_idx = 0
    plan_end_idx = 4955

    lanes_start_idx = plan_end_idx
    lanes_end_idx = lanes_start_idx + 528

    lane_lines_prob_start_idx = lanes_end_idx
    lane_lines_prob_end_idx = lane_lines_prob_start_idx + 8

    road_start_idx = lane_lines_prob_end_idx
    road_end_idx = road_start_idx + 264

    plan = res[:, plan_start_idx:plan_end_idx]  # (N, 4955)
    paths = np.array(np.split(plan, 5, axis=1)).reshape(-1, 5, 991)  # (N, 5, 991)
    path_probs = paths[:, :, -1]  # (N, 5)
    paths = paths[:, :, :-1].reshape(-1, 5, 2, 33, 15)  # (N, 5, 2, 33, 15)

    best_idx = np.argmax(path_probs, axis=1)[0]  # (N,)
    best_path = paths[:, best_idx, ...]  # (N, 2, 33, 15)

    lanes = res[:, lanes_start_idx:lanes_end_idx]
    lane_road = res[:, road_start_idx:road_end_idx]

    ll_t = lanes[:, 0:66]

    ll_t2 = lanes[:, 66:132]
    outer_left_lane, outer_left_lane_std = seperate_points_and_std_values(ll_t)
    inner_left_lane, inner_left_lane_std = seperate_points_and_std_values(ll_t2)

    l_t = lanes[:, 132:198]
    l_t2 = lanes[:, 198:264]
    outer_right_lane, outer_right_lane_std = seperate_points_and_std_values(l_t2)
    inner_right_lane, inner_right_lane_std = seperate_points_and_std_values(l_t)

    roadr_t = lane_road[:, 0:66]
    roadr_t2 = lane_road[:, 66:132]
    left_road_edge, left_road_edge_std = seperate_points_and_std_values(roadr_t)
    right_road_edge, right_road_edge_std = seperate_points_and_std_values(roadr_t2)

    # lanelines with std
    lanelines = [
        (left_road_edge, left_road_edge_std, 'yellow'),
        # (outer_left_lane,  outer_left_lane_std, 'sandybrown'),
        (inner_left_lane,  inner_left_lane_std, 'white'),
        (inner_right_lane, inner_right_lane_std, 'white'),
        # (outer_right_lane, outer_right_lane_std, 'sandybrown'),
        (right_road_edge, right_road_edge_std, 'yellow'),
    ]

    return lanelines, best_path

In [4]:
class Calibration:
  def __init__(self, rpy, intrinsic=eon_intrinsics):
    self.intrinsic = intrinsic
    self.extrinsics_matrix = get_view_frame_from_calib_frame(rpy[0], rpy[1], rpy[2], 0)[:,:3]
    self.zoom = _CALIB_BB_TO_FULL[0, 0]

  def car_space_to_ff(self, x, y, z):
    car_space_projective = np.column_stack((x, y, z)).T
    ep = self.extrinsics_matrix.dot(car_space_projective)
    kep = self.intrinsic.dot(ep)
    return (kep[:-1, :] / kep[-1, :]).T

  def car_space_to_bb(self, x, y, z):
    pts = self.car_space_to_ff(x, y, z)
    return pts / self.zoom


def project_path(path, calibration, z_off=1.22):
  '''Projects paths from calibration space (model input/output) to image space.'''

  x = path[:, 0]
  y = path[:, 1]
  z = path[:, 2] + z_off
  pts = calibration.car_space_to_bb(x, y, z)
  pts[pts < 0] = np.nan
  valid = np.isfinite(pts).all(axis=1)
  pts = pts[valid].astype(int)

  return pts


def draw_path(calib_path, img, calibration, width=1, height=1.2, fill_color=(128,0,255), line_color=(0,255,0)):
  calib_path_l = calib_path - np.array([0, width, 0])
  calib_path_r = calib_path + np.array([0, width, 0])

  img_pts_l = project_path(calib_path_l, calibration, z_off=height)
  img_pts_r = project_path(calib_path_r, calibration, z_off=height)

  for i in range(1, len(img_pts_l)):
    u1,v1,u2,v2 = np.append(img_pts_l[i-1], img_pts_r[i-1])
    u3,v3,u4,v4 = np.append(img_pts_l[i], img_pts_r[i])
    pts = np.array([[u1,v1],[u2,v2],[u4,v4],[u3,v3]], np.int32).reshape((-1,1,2))
    cv2.fillPoly(img,[pts],fill_color)
    cv2.polylines(img,[pts],True,line_color)

In [5]:
real_preds = np.load('/home/nikita/data/2021-09-14--09-19-21/25/xyz_preds.npz')
real_paths_xyz = np.stack((real_preds['x'], real_preds['y'], real_preds['z']), axis=-1)
real_paths_xyz.shape

(1200, 33, 3)

In [6]:
comma_recordings_basedir = '/home/nikita/data'
path_to_onnx_model = 'train/supercombo.onnx'
os.makedirs('outs', exist_ok=True)

train_split = 0.8
seq_len = 1001
single_frame_batches = False
prefetch_factor = 1
batch_size = 1
num_workers = 1

train_dataset = CommaDataset(comma_recordings_basedir, train_split=train_split, seq_len=seq_len,
                                shuffle=True, single_frame_batches=single_frame_batches, seed=42)
train_loader = DataLoader(train_dataset, batch_size=None, num_workers=num_workers, shuffle=False,
                        prefetch_factor=prefetch_factor, persistent_workers=True, collate_fn=None)
train_loader = BatchDataLoader(train_loader, batch_size=batch_size)
train_loader = BackgroundGenerator(train_loader)

model = onnx.load(path_to_onnx_model)

input_names = [node.name for node in model.graph.input]
output_names = [node.name for node in model.graph.output]

providers = ['CPUExecutionProvider']
onnxruntime_model = rt.InferenceSession(path_to_onnx_model, providers=providers)

recurrent_state = np.zeros((1, 512), dtype=np.float32)

for batch in train_loader:
    stacked_frames, plans, plans_probs, segment_finished, sequence_finished, bgr_frames = batch

    for t_idx in range(seq_len):

        input_frame = stacked_frames[0:1, t_idx, ...].numpy().astype(np.float32)
        bgr_frame = bgr_frames[0, t_idx, ...].numpy()

        inputs = {
            'input_imgs': input_frame,
            'desire': np.zeros((1, 8), dtype=np.float32),
            'traffic_convention': np.array([0, 1], dtype=np.float32).reshape(1, 2),
            'initial_state': recurrent_state,
        }

        outs = onnxruntime_model.run(output_names, inputs)[0]
        lanelines, best_path = extract_preds(outs)
        recurrent_state = outs[:, -512:]

        if t_idx % 100 == 0:
            print('t_idx:', t_idx)
            # plot predictions
            rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)
            img_gt = np.zeros((plot_img_height, plot_img_width, 3), dtype='uint8')
            zoom_matrix = _BB_TO_FULL_FRAME
            cv2.warpAffine(rgb_frame, zoom_matrix[:2], (img_gt.shape[1], img_gt.shape[0]), dst=img_gt, flags=cv2.WARP_INVERSE_MAP)
            img_pred = img_gt.copy()

            rpy_calib_gt = [0.00018335809, 0.034165092, -0.014245722]  # real calibration values during this ride
            rpy_calib_pred = [0, 0, 0]  # calibration we currently use for pre-processing

            calibration_gt = Calibration(rpy_calib_gt)
            calibration_pred = Calibration(rpy_calib_pred)

            path_xyz_gt = real_paths_xyz[t_idx]
            path_xyz_pred = best_path[0, 0, :, :3]

            draw_path(path_xyz_gt, img_gt, calibration_gt)
            draw_path(path_xyz_pred, img_pred, calibration_pred)

            plot_width = 15
            plot_height = plot_width * (height / width)
            plt.figure(figsize=(plot_width, plot_height))

            plt.imsave(f'outs/{t_idx}-gt.png', img_gt)
            plt.imsave(f'outs/{t_idx}-pred.png', img_pred)
            plt.close()

    break

Using cached paths to videos and GTs...
Total # segments 72
Subset # segments: 58
t_idx: 0
t_idx: 100
t_idx: 200
t_idx: 300
t_idx: 400
t_idx: 500
t_idx: 600
t_idx: 700
t_idx: 800
t_idx: 900
t_idx: 1000
